# Constants

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
print("Environment variables have been loaded ;-)")

deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME_MINI"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')
version = importlib.metadata.version("azure-ai-projects")
print(f"azure-ai-projects library installed version: {version}")

Environment variables have been loaded ;-)
Project Connection String: <...hub01-grp;mmai-swc-hub01-prj01>
azure-ai-projects library installed version: 1.0.0b10


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    FileSearchTool,
)
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai-swc-hub01-grp',
 'project_name': 'mmai-swc-hub01-prj01'}

# Check if we have any files already on Azure AI Foundry
For better clarity, we delete all pre-existing files, if present

In [3]:
# delete all files

# the helper function below implements the call 
# project_client.agents.list_files()['data']

i=0
for file in list_all_files(project_client)['content']['data']:
    i += 1
    project_client.agents.delete_file(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



# Upload file(s) to search into

In [4]:
file = project_client.agents.upload_file_and_poll(file_path="./product_info_1.md", purpose="assistants")
print(f"Uploaded file: {file}")

Uploaded file: {'object': 'file', 'id': 'assistant-Ks3xwWEJE1oTvq1b38mHSf', 'purpose': 'assistants', 'filename': 'product_info_1.md', 'bytes': 10999, 'created_at': 1748330390, 'expires_at': None, 'status': 'processed', 'status_details': None}


# Create a vector store to vectorize file(s)

In [5]:
vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f'Created vector store <{vector_store}>\n\n...and uploaded file "{file.filename}".')

Created vector store <{'id': 'vs_OdddUr5FlWLjPkWdFtrvobjU', 'object': 'vector_store', 'name': 'my_vectorstore', 'status': 'completed', 'usage_bytes': 17143, 'created_at': 1748330390, 'file_counts': {'in_progress': 0, 'completed': 1, 'failed': 0, 'cancelled': 0, 'total': 1}, 'metadata': {}, 'expires_after': None, 'expires_at': None, 'last_active_at': 1748330390}>

...and uploaded file "product_info_1.md".


# Create file search tool with resources followed by creating agent

In [6]:
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

print(f"file_search.definitions: {file_search_tool.definitions}")
print(f"file_search.resources: {file_search_tool.resources}")

file_search.definitions: [{'type': 'file_search'}]
file_search.resources: {'file_search': {'vector_store_ids': ['vs_OdddUr5FlWLjPkWdFtrvobjU']}}


# Create AI Foundry Agent

In [7]:
agent = project_client.agents.create_agent(
    model=deployment_name,
    name="file-agent",
    instructions="Hello, you are helpful agent that can search information from uploaded files",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)

print(f"Created AI Foundry agent, ID: {agent.id}")

Created AI Foundry agent, ID: asst_Lk8AozMyFalIJj8Fl5aXgJIg


# Create the thread and attach a new message to it

In [8]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Hello, how much for the TrailMaster?",
)
print(f"The following message has been attached to the thread <{thread.id}>:\n{message} ")

Created thread: {'id': 'thread_grxFF19r13uv5eCGbR8DYOb4', 'object': 'thread', 'created_at': 1748330392, 'metadata': {}, 'tool_resources': {}}

The following message has been attached to the thread <thread_grxFF19r13uv5eCGbR8DYOb4>:
{'id': 'msg_awEC5hFeedvuvCKLJIPxcWst', 'object': 'thread.message', 'created_at': 1748330393, 'assistant_id': None, 'thread_id': 'thread_grxFF19r13uv5eCGbR8DYOb4', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, how much for the TrailMaster?', 'annotations': []}}], 'attachments': [], 'metadata': {}} 


# Run the agent syncrhonously

In [9]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_LUtnkPQQ0GkDdnY3XWUis6G3', 'object': 'thread.run', 'created_at': 1748330393, 'assistant_id': 'asst_Lk8AozMyFalIJj8Fl5aXgJIg', 'thread_id': 'thread_grxFF19r13uv5eCGbR8DYOb4', 'status': 'completed', 'started_at': 1748330393, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1748330394, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': 'Hello, you are helpful agent that can search information from uploaded files', 'tools': [{'type': 'file_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 6066, 'completion_tokens': 43, 'total_tokens': 6109, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': 

# Fetch messages from the thread after the agent run execution

In [10]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: Hello, how much for the TrailMaster?

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: The price for the TrailMaster X4 Tent is **$250**【4:2†source】.
>>> Annotation in MessageTextContent 1 of message 2: 【4:2†source】



# Fetch citations from the messages

In [11]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path citations: {len(messages.file_citation_annotations)}\n")

i=0
for file_citation_annotation in messages.file_citation_annotations:
    i += 1
    print(f"{i} - File file_citation_annotation: {file_citation_annotation}")

Nr. of file path citations: 1

1 - File file_citation_annotation: {'type': 'file_citation', 'text': '【4:2†source】', 'start_index': 49, 'end_index': 61, 'file_citation': {'file_id': 'assistant-Ks3xwWEJE1oTvq1b38mHSf'}}


# Retrieve and download eventual images

In [12]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path annotations: {len(messages.file_path_annotations)}\n")

i=0
for file_path_annotation in messages.file_path_annotations:
    i += 1
    print(f"{i} - File annotation paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}\\{file_name}")
    image = Image.open(f"{os.getcwd()}\\{file_name}") # Open the image
    image.show() # Display the image

Nr. of file path annotations: 0



# Collect all resources for this project

In [13]:
all_agents = list_all_agents(project_client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(project_client)
print(all_threads["summary"])

all_files = list_all_files(project_client)
print(all_files["summary"])

all_runs = list_all_runs(project_client)
print(all_runs["summary"])

all_runsteps=list_all_runsteps(project_client)
print(all_runsteps["summary"])

all_messages = list_all_messages(project_client)
print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(project_client=project_client)
print(all_vectorstores["summary"])

1 agents in project <mmai-swc-hub01-prj01>
1 threads in project <mmai-swc-hub01-prj01>
1 files in project <mmai-swc-hub01-prj01>
1 runs in 1 threads of project <mmai-swc-hub01-prj01>
2 run steps in 1 pairs of (thread, run) of project <mmai-swc-hub01-prj01>
2 messages in 1 threads of project <mmai-swc-hub01-prj01>
1 vector stores in project <mmai-swc-hub01-prj01>


# Teardown for all resources

In [14]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]["data"]:
    i += 1
    project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(project_client=project_client)

print(f"Vector stores deleted: {i}\n")

1 - Vector store <vs_OdddUr5FlWLjPkWdFtrvobjU> has been deleted
Vector stores deleted: 1



In [15]:
# delete all files

i=0
for file in all_files['content']['data']:
    i += 1
    project_client.agents.delete_file(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

1 - File <product_info_1.md> (assistant-Ks3xwWEJE1oTvq1b38mHSf) has been deleted
Files deleted: 1



In [16]:
# delete all threads

i=0
for thread in all_threads["content"]["data"]:
    i += 1
    project_client.agents.delete_thread(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_grxFF19r13uv5eCGbR8DYOb4> has been deleted
Threads deleted: 1



In [17]:
# delete all agents

i=0
for agent in all_agents["content"]["data"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(project_client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_Lk8AozMyFalIJj8Fl5aXgJIg> has been deleted
Agents deleted: 1



# HIC SUNT LEONES